In [9]:
!pip install d2l==1.0.0a1.post0

In [10]:
!pip install torchmetrics

In [11]:
import os

import joblib as jb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from d2l import torch as d2l
from torch import nn
from torchmetrics import R2Score
from torchsummary import summary

In [12]:

class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.

    The separable convlution is a method to reduce number of the parameters 
    in the deep learning network for slight decrease in predictions quality.
    """

    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(
            ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))


In [13]:
class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.

    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """

    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):

        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


In [14]:

class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)


In [15]:

class Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()

        self.net = nn.Sequential(
            # nn.Flatten(),
            nn.Linear(raw_ni, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, no),
        )

    def forward(self, x):
        out = self.net(x)
        return out


In [16]:
# Calculating the loss
# loss = nn.CrossEntropyLoss(reduction='none')
loss = nn.MSELoss(reduction='none')


def evaluate_loss(data_iter, net, devices):
    l_sum, n = 0.0, 0
    for features, labels in data_iter:
        features, labels = features.to(devices[0]), labels.to(devices[0])
        outputs = net(features)
        l = loss(outputs, labels)
        l_sum += l.sum()
        n += labels.numel()
    return l_sum / n


In [17]:
def loss_plotter(train_losses,test_losses):
    plt.figure(figsize = (10,6) )
    plt.plot(train_losses, 'r*-', label = "Train Loss")
    plt.plot(test_losses, 'b*-', label = "Test Loss")
    plt.legend()
    plt.grid()
    plt.xlabel("Epochs")
    plt.ylabel("The Loss Value")
    
    

In [18]:
spo2_vals = jb.load('strided_dataset_spo2_vals.pk')

In [19]:
stride_dataset = jb.load('strided_dataset.pk')

In [20]:
dataset = pd.DataFrame.from_dict(stride_dataset).transpose()


In [21]:
dataset_spo2 = pd.DataFrame.from_dict(spo2_vals).transpose()

In [22]:
dataset_spo2 = dataset_spo2/100

In [23]:
X_train, X_valid, X_test = np.split(dataset.sample(
    frac=1), [int(.8*len(dataset)), int(.9*len(dataset))])
y_train, y_valid, y_test = dataset_spo2.loc[X_train.index], dataset_spo2.loc[
    X_valid.index], dataset_spo2.loc[X_test.index]


In [24]:
# unroll the data
X_train_unrolled = []
y_train_unrolled = []
for r in X_train.index:
    for c in X_train.columns:
        # skip values containing NaN
        if not np.isnan(X_train.loc[r][c]).any() and not np.isnan(y_train.loc[r][c]):
            X_train_unrolled.append(X_train.loc[r][c])
            y_train_unrolled.append(y_train.loc[r][c])

X_valid_unrolled = []
y_valid_unrolled = []
for r in X_valid.index:
    for c in X_valid.columns:
        # skip values containing NaN
        if not np.isnan(X_valid.loc[r][c]).any() and not np.isnan(y_valid.loc[r][c]):
            X_valid_unrolled.append(X_valid.loc[r][c])
            y_valid_unrolled.append(y_valid.loc[r][c])

X_test_unrolled = []
y_test_unrolled = []
for r in X_test.index:
    for c in X_test.columns:
        # skip values containing NaN
        if not np.isnan(X_test.loc[r][c]).any() and not np.isnan(y_test.loc[r][c]):
            X_test_unrolled.append(X_test.loc[r][c])
            y_test_unrolled.append(y_test.loc[r][c])


In [25]:
# convert to tensors
X_train, y_train = torch.tensor(X_train_unrolled, dtype=torch.float32), torch.tensor(
    y_train_unrolled, dtype=torch.float32)
X_valid, y_valid = torch.tensor(X_valid_unrolled, dtype=torch.float32), torch.tensor(
    y_valid_unrolled, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test_unrolled, dtype=torch.float32), torch.tensor(
    y_test_unrolled, dtype=torch.float32)

C:\Users\Owner\AppData\Local\Temp\ipykernel_17560\997760739.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  X_train, y_train = torch.tensor(X_train_unrolled, dtype=torch.float32), torch.tensor(


In [26]:
# reshape y tensors
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [27]:
# normalize x data
X_train_mean = X_train.mean() 
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / (X_train_std + 1e-7)

X_valid_mean = X_valid.mean()
X_valid_std = X_valid.std()
X_valid = (X_valid - X_valid_mean) / (X_valid_std + 1e-7)

X_test_mean = X_test.mean()
X_test_std = X_test.std()
X_test = (X_test - X_test_mean) / (X_test_std + 1e-7)

In [28]:
# convert to torch iterators
batch_size = 64
train_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_train, y_train), batch_size=batch_size, shuffle=True)
valid_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_valid, y_valid), batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_test, y_test), batch_size=batch_size, shuffle=True)

In [29]:

num_epochs = 10
lr, wd = 0.001, 1e-4
lr_period, lr_decay = 2, 0.9


In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
model = Classifier(240,1).to(device)
#Setting the loss function
cost = nn.L1Loss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#this is defined to print how many steps are remaining when training
total_step = len(train_iter)

In [32]:
summary(model,(1,240))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 128]          30,848
              ReLU-2               [-1, 1, 128]               0
            Linear-3                [-1, 1, 64]           8,256
              ReLU-4                [-1, 1, 64]               0
            Linear-5                [-1, 1, 32]           2,080
              ReLU-6                [-1, 1, 32]               0
            Linear-7                 [-1, 1, 1]              33
Total params: 41,217
Trainable params: 41,217
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.16
Estimated Total Size (MB): 0.16
----------------------------------------------------------------


In [33]:
train_losses = []
test_losses = []

num_epochs = 500
for epoch in range(num_epochs):
    

    
    
    for i, (features, labels) in enumerate(train_iter):
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features)
        loss = cost(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
        
    for i, (features_test, labels_test) in enumerate(test_iter):
        features_test = features_test.to(device)
        labels_test = labels_test.to(device)
        outputs_test = model(features_test)
        loss_test = cost(outputs_test,labels_test)
     
    train_losses.append(loss.item()*100)
    test_losses.append(loss_test.item()*100)
    print("1) The Train Loss value with the epoch: " + str(epoch) + " is equal to " + str(loss.item()*100))
    print("2) The Test Loss value with the epoch: " + str(epoch) + " is equal to " + str(loss_test.item()*100))
    print("-------------------------------------------------------------------")
    
    if epoch == num_epochs-1:
        break



1) The Train Loss value with the epoch: 0 is equal to 3.419756516814232
2) The Test Loss value with the epoch: 0 is equal to 32.940101623535156
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 1 is equal to 3.2345090061426163
2) The Test Loss value with the epoch: 1 is equal to 34.087976813316345
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 2 is equal to 2.946598455309868
2) The Test Loss value with the epoch: 2 is equal to 30.828896164894104
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 3 is equal to 2.4427341297268867
2) The Test Loss value with the epoch: 3 is equal to 28.03359627723694
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 4 is equal to 3.033946640789509
2) The Test Loss value with the epoch: 4 is equal to 23.255722224712372
-------

1) The Train Loss value with the epoch: 39 is equal to 1.910896971821785
2) The Test Loss value with the epoch: 39 is equal to 3.026524744927883
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 40 is equal to 1.09757911413908
2) The Test Loss value with the epoch: 40 is equal to 3.817615658044815
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 41 is equal to 1.3379232957959175
2) The Test Loss value with the epoch: 41 is equal to 4.355844110250473
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 42 is equal to 1.8193842843174934
2) The Test Loss value with the epoch: 42 is equal to 3.374844789505005
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 43 is equal to 1.0616474784910679
2) The Test Loss value with the epoch: 43 is equal to 3.732646629214287
-

1) The Train Loss value with the epoch: 78 is equal to 0.9344147518277168
2) The Test Loss value with the epoch: 78 is equal to 3.2096009701490402
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 79 is equal to 1.2981012463569641
2) The Test Loss value with the epoch: 79 is equal to 2.9049884527921677
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 80 is equal to 1.396748423576355
2) The Test Loss value with the epoch: 80 is equal to 3.1911350786685944
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 81 is equal to 1.2747799046337605
2) The Test Loss value with the epoch: 81 is equal to 2.9199521988630295
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 82 is equal to 0.79917311668396
2) The Test Loss value with the epoch: 82 is equal to 4.2449939996004

1) The Train Loss value with the epoch: 117 is equal to 1.0249472223222256
2) The Test Loss value with the epoch: 117 is equal to 3.2119862735271454
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 118 is equal to 1.020371913909912
2) The Test Loss value with the epoch: 118 is equal to 2.9418695718050003
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 119 is equal to 1.610439456999302
2) The Test Loss value with the epoch: 119 is equal to 2.035347744822502
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 120 is equal to 1.0477627627551556
2) The Test Loss value with the epoch: 120 is equal to 5.168092250823975
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 121 is equal to 2.0838961005210876
2) The Test Loss value with the epoch: 121 is equal to 3.4156

1) The Train Loss value with the epoch: 155 is equal to 1.1995675042271614
2) The Test Loss value with the epoch: 155 is equal to 4.229611158370972
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 156 is equal to 1.2866662815213203
2) The Test Loss value with the epoch: 156 is equal to 4.120359569787979
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 157 is equal to 1.1281507089734077
2) The Test Loss value with the epoch: 157 is equal to 3.46716046333313
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 158 is equal to 1.039606984704733
2) The Test Loss value with the epoch: 158 is equal to 4.944602772593498
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 159 is equal to 1.1678018607199192
2) The Test Loss value with the epoch: 159 is equal to 4.355590

1) The Train Loss value with the epoch: 193 is equal to 0.7226946298032999
2) The Test Loss value with the epoch: 193 is equal to 4.154770448803902
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 194 is equal to 1.2460450641810894
2) The Test Loss value with the epoch: 194 is equal to 4.756028205156326
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 195 is equal to 0.7260478101670742
2) The Test Loss value with the epoch: 195 is equal to 2.903006039559841
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 196 is equal to 1.0012894868850708
2) The Test Loss value with the epoch: 196 is equal to 3.5308711230754852
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 197 is equal to 1.0132508352398872
2) The Test Loss value with the epoch: 197 is equal to 4.975

KeyboardInterrupt: 

In [ ]:
loss_plotter(train_losses,test_losses)